In [ ]:
#necessary installs

!pip install tensorflow
!pip install datasets
!pip install transformers
!pip install datasets
!pip install  scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.4 MB/s eta 0:00:00


In [ ]:
#from
from google.colab import drive
from huggingface_hub import notebook_login
from torch.utils.data import DataLoader, TensorDataset
from datasets import Dataset
from transformers import AutoTokenizer, BertModel, DistilBertModel, DistilBertTokenizer
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score

#imports
import pandas as pd
import tensorflow as tf
import re
import numpy as np
import matplotlib.pyplot as plt
import warnings
import concurrent.futures
import torch
import time
import sys
import torch.nn as nn
import torch.optim as optim

In [ ]:
#paths
drive.mount('/content/drive')
notebook_login()

dataPATH = '/content/drive/MyDrive/LINKUP_DATA'

trainPATH = f"{dataPATH}/train.csv"

testPATH=f"{dataPATH}/test.csv"

#Load data sets

df_train = pd.read_csv(trainPATH)

df_test = pd.read_csv(testPATH)

#checking shape

print("Training shape: ",df_train.shape) #expected 8: id, text, and 6 categories

print("Test shape: ",df_test.shape) #expected 2: id & text



Mounted at /content/drive


Training shape:  (159571, 8)
Test shape:  (153164, 2)


In [ ]:

#Load Tokenizer & Device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}", flush=True)

# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
# bert_model = BertModel.from_pretrained("bert-base-uncased").to(device)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)
bert_model = DistilBertModel.from_pretrained("distilbert-base-uncased").to(device)

#Balance Dataset Using Oversampling
def balance_dataset(df):
    label_columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    df_majority = df[df[label_columns].sum(axis=1) == 0]

    balanced_dfs = [df_majority]
    for label in label_columns:
        df_minority = df[df[label] == 1]
        df_minority_upsampled = resample(df_minority, replace=True,
                                         n_samples=len(df_majority) // len(label_columns),
                                         random_state=42)
        balanced_dfs.append(df_minority_upsampled)

    df_balanced = pd.concat(balanced_dfs).sample(frac=1, random_state=42).reset_index(drop=True)
    return df_balanced

df_train_balanced = balance_dataset(df_train)
print("New Label Distribution:\n", df_train_balanced.iloc[:, 2:].sum(), flush=True)

#Convert Labels to Multi-Hot Encoded Vectors
label_columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y_train = df_train_balanced[label_columns].values

#Split Data into Train & Validation Sets
X_train, X_val, y_train, y_val = train_test_split(df_train_balanced["comment_text"], y_train,
                                                  test_size=0.2, random_state=42)

print(f"Training Samples: {len(X_train)}, Validation Samples: {len(X_val)}", flush=True)

# Convert Text to Tokenized Tensors
def tokenize_texts(texts, tokenizer, max_length=128):
    encodings = tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    return encodings["input_ids"], encodings["attention_mask"]

# def tokenize_texts(texts, tokenizer, max_length=128, batch_size=50000):
#     input_ids_list = []
#     attention_masks_list = []

#     start_time = time.time()

#     for i in range(0, len(texts), batch_size):
#         batch_texts = texts[i:i+batch_size]
#         encodings = tokenizer(
#             list(batch_texts),
#             padding="max_length",
#             truncation=True,
#             max_length=max_length,
#             return_tensors="pt"
#         )
#         input_ids_list.append(encodings["input_ids"])
#         attention_masks_list.append(encodings["attention_mask"])

#         elapsed_time = time.time() - start_time
#         print(f"Processed {i + len(batch_texts)} / {len(texts)} samples - Time elapsed: {elapsed_time:.2f} sec", flush=True)

#     print(f"\nTokenization complete! Total time: {time.time() - start_time:.2f} sec", flush=True)

#     return torch.cat(input_ids_list, dim=0), torch.cat(attention_masks_list, dim=0)


#Tokenize Training and Validation Sets
X_train_tokens, X_train_masks = tokenize_texts(X_train, tokenizer)
X_val_tokens, X_val_masks = tokenize_texts(X_val, tokenizer)
print("Tokenization Complete", flush=True)

#Convert labels to tensors
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# Create DataLoader for Batches
batch_size = 128

train_dataset = TensorDataset(X_train_tokens, X_train_masks, y_train_tensor)
val_dataset = TensorDataset(X_val_tokens, X_val_masks, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)  # Disable multiprocessing for debugging
val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=0)

print("DataLoader Initialized", flush=True)

# # Multi-Label Classification Model
# class MultiLabelClassifier(nn.Module):
#     def __init__(self, output_dim):
#         super(MultiLabelClassifier, self).__init__()
#         self.bert = BertModel.from_pretrained("bert-base-uncased")  # Load BERT model
#         self.fc = nn.Linear(self.bert.config.hidden_size, output_dim)
#         self.sigmoid = nn.Sigmoid()  # Multi-label classification

#     def forward(self, input_ids, attention_mask):
#         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
#         pooled_output = outputs.pooler_output  # Get pooled BERT representation
#         return self.sigmoid(self.fc(pooled_output))  # Convert logits to probabilities

class MultiLabelClassifier(nn.Module):
    def __init__(self, output_dim):
        super(MultiLabelClassifier, self).__init__()
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.fc = nn.Linear(self.bert.config.hidden_size, output_dim)
        self.sigmoid = nn.Sigmoid()  # Multi-label classification

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        return self.sigmoid(self.fc(pooled_output))  # Convert logits to probabilities


# Model Setup
output_dim = y_train_tensor.shape[1]  # Number of labels
model = MultiLabelClassifier(output_dim).to(device)

criterion = nn.BCELoss()  # Binary cross-entropy for multi-label classification
optimizer = optim.Adam(model.parameters(), lr=2e-5)

#Debugging Assertion to Confirm Execution Reaches This Point
print("\nModel and Optimizer Initialized!", flush=True)



Using device: cuda
New Label Distribution:
 toxic            136995
severe_toxic      45658
obscene          110301
threat            29616
insult           108440
identity_hate     41943
dtype: int64
Training Samples: 229353, Validation Samples: 57339
Tokenization Complete
DataLoader Initialized

Model and Optimizer Initialized!


In [ ]:
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score

# Define Class Labels for Better Display
label_names = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

epochs = 3

for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs} starting...", flush=True)

    model.train()
    total_loss = 0
    all_labels = []
    all_outputs = []

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=True)

    for batch_idx, batch in enumerate(progress_bar):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        #Store outputs & labels for metric calculation
        all_labels.append(labels.cpu().detach().numpy())
        all_outputs.append(outputs.cpu().detach().numpy())

        #Calculate metrics after each batch
        labels_np = np.vstack(all_labels)
        outputs_np = np.vstack(all_outputs).round()

        auc_score = roc_auc_score(labels_np, outputs_np, average="macro")
        f1 = f1_score(labels_np, outputs_np, average="macro")
        precision = precision_score(labels_np, outputs_np, average="macro", zero_division=0)
        recall = recall_score(labels_np, outputs_np, average="macro", zero_division=0)

        #Update the progress bar with metrics
        progress_bar.set_postfix({
            "Loss": loss.item(),
            "AUC": f"{auc_score:.4f}",
            "F1": f"{f1:.4f}",
            "Precision": f"{precision:.4f}",
            "Recall": f"{recall:.4f}"
        })

    avg_train_loss = total_loss / len(train_loader)

    #Compute Final Metrics for the Epoch
    all_labels = np.vstack(all_labels)
    all_outputs = np.vstack(all_outputs)

    auc_score = roc_auc_score(all_labels, all_outputs, average="macro")
    f1 = f1_score(all_labels, all_outputs.round(), average="macro")
    precision = precision_score(all_labels, all_outputs.round(), average="macro", zero_division=0)
    recall = recall_score(all_labels, all_outputs.round(), average="macro", zero_division=0)

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | "
          f"AUC: {auc_score:.4f} | F1-score: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}",
          flush=True)

print("\nModel Training Complete!", flush=True)



Epoch 1/3 starting...


Epoch 1/3: 100%|██████████| 1792/1792 [21:33<00:00,  1.39it/s, Loss=0.0594, AUC=0.9377, F1=0.9119, Precision=0.9185, Recall=0.9060]


Epoch 1/3 | Train Loss: 0.1065 | AUC: 0.9896 | F1-score: 0.9119 | Precision: 0.9185 | Recall: 0.9060

Epoch 2/3 starting...


Epoch 2/3: 100%|██████████| 1792/1792 [21:32<00:00,  1.39it/s, Loss=0.0245, AUC=0.9875, F1=0.9829, Precision=0.9807, Recall=0.9852]


Epoch 2/3 | Train Loss: 0.0346 | AUC: 0.9987 | F1-score: 0.9829 | Precision: 0.9807 | Recall: 0.9852

Epoch 3/3 starting...


Epoch 3/3: 100%|██████████| 1792/1792 [21:27<00:00,  1.39it/s, Loss=0.00776, AUC=0.9935, F1=0.9915, Precision=0.9907, Recall=0.9923]


Epoch 3/3 | Train Loss: 0.0185 | AUC: 0.9995 | F1-score: 0.9915 | Precision: 0.9907 | Recall: 0.9923

Model Training Complete!


In [ ]:
import os
import torch

model_save_dir = "/content/drive/MyDrive/LINKUP_modelDIS"
os.makedirs(model_save_dir, exist_ok=True)

model_save_path = os.path.join(model_save_dir, "LINKUP_modelDIS.pth")
torch.save(model, model_save_path)
print(f" Model saved at {model_save_path}")





In [ ]:
#Load Model
model = torch.load(model_save_path)
model.to(device)
model.eval()
print("Model loaded")

#Tokenize Test Data
X_test_tokens, X_test_masks = tokenize_texts(df_test["comment_text"], tokenizer)
print("Tokenization Complete")

# Test DataLoader
batch_size = 128
test_dataset = TensorDataset(X_test_tokens, X_test_masks)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Run Model on Test Data

all_predictions = []
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids, attention_mask = [x.to(device) for x in batch]
        outputs = model(input_ids, attention_mask)
        all_predictions.append(outputs.cpu().numpy())

# Convert Predictions to Labels
all_predictions = np.vstack(all_predictions)
threshold = 0.5
binary_predictions = (all_predictions >= threshold).astype(int)

#Save Predictions
label_columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
df_test_results = df_test.copy()
df_test_results[label_columns] = binary_predictions

output_path = "/content/drive/MyDrive/LINKUP_model2/test_predictions2.csv"
df_test_results.to_csv(output_path, index=False)
print(f"\n Predictions saved to: {output_path}")


<ipython-input-7-62c5b61d5126>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_save_path)


Model loaded
Tokenization Complete


100%|██████████| 1197/1197 [03:22<00:00,  5.91it/s]



 Predictions saved to: /content/drive/MyDrive/LINKUP_model2/test_predictions2.csv


In [ ]:
import torch
from transformers import AutoTokenizer

# Define Device (Use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Model
model_save_path = "/content/drive/MyDrive/LINKUP_model2/LINKUP_model2.pth"
model = torch.load(model_save_path, map_location=device)
model.to(device)
model.eval()
print("Model loaded successfully!")

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print("Tokenizer loaded!")

# Define Sample Test Sentences
sample_texts = [
    "You are the worst person ever!",  # Likely toxic, insult
    "I hope you have an amazing day!",  # Not toxic
    "I'll kill u!",  # Likely threat
    "This is absolute garbage!",  # Possibly toxic, insult
    "What a wonderful place to visit!",  # Not toxic
    "I love you",  # Should not be toxic
    "Hello, my name is Anthony",  # Should not be toxic

]

# Function to Tokenize Input Texts
def tokenize_texts(texts, tokenizer, max_length=128):
    encoded = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    return encoded["input_ids"], encoded["attention_mask"]

# Tokenize Sample Inputs
X_sample_tokens, X_sample_masks = tokenize_texts(sample_texts, tokenizer)
X_sample_tokens, X_sample_masks = X_sample_tokens.to(device), X_sample_masks.to(device)
print("Tokenization complete! Shape:", X_sample_tokens.shape)

# Run Model on Sample Texts
with torch.no_grad():
    outputs = model(X_sample_tokens, attention_mask=X_sample_masks)  # Get raw logits
    predictions = torch.sigmoid(outputs).cpu().numpy()  # Apply sigmoid activation

print("Processed Predictions:", predictions)  # Debugging Step

# Adjust Threshold to Reduce False Positives
threshold = 0.6  # Test a higher threshold for better classification
binary_predictions = (predictions >= threshold).astype(int)

# Define Label Categories
label_columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

# Print Results
for i, text in enumerate(sample_texts):
    predicted_labels = [label_columns[j] for j in range(len(label_columns)) if binary_predictions[i][j] == 1]
    print(f"\nText: {text}")
    print("Predicted Categories:", predicted_labels if predicted_labels else ["None"])




<ipython-input-9-bd40fb1f17fb>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_save_path, map_location=device)


Model loaded successfully!
Tokenizer loaded!
Tokenization complete! Shape: torch.Size([7, 128])
Processed Predictions: [[0.70951307 0.50007236 0.5000425  0.50091904 0.62429357 0.5000412 ]
 [0.5005124  0.5000309  0.5000346  0.50003064 0.5001662  0.50002295]
 [0.730574   0.50017583 0.50074923 0.73093516 0.5006667  0.5006944 ]
 [0.730562   0.50003284 0.6329238  0.50002927 0.50415385 0.5000302 ]
 [0.50018156 0.5000454  0.5000169  0.50004464 0.5000871  0.5000303 ]
 [0.50049883 0.5000363  0.5000093  0.50471526 0.5002195  0.50002277]
 [0.50015575 0.50004303 0.5000139  0.500046   0.50004935 0.5000331 ]]

Text: You are the worst person ever!
Predicted Categories: ['toxic', 'insult']

Text: I hope you have an amazing day!
Predicted Categories: ['None']

Text: I'll kill u!
Predicted Categories: ['toxic', 'threat']

Text: This is absolute garbage!
Predicted Categories: ['toxic', 'obscene']

Text: What a wonderful place to visit!
Predicted Categories: ['None']

Text: I love you
Predicted Categories

In [ ]:
import torch
from transformers import DistilBertModel, DistilBertTokenizerFast

# Load the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# Load the fine-tuned model
model_save_path = "/content/drive/MyDrive/LINKUP_modelDIS/LINKUP_modelDIS.pth"
model = torch.load(model_save_path)
model.eval()  # Set to evaluation mode before further training


<ipython-input-5-dbffb481cfdc>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_save_path)


MultiLabelClassifier(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): L

In [ ]:
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)  # Reduce learning rate for fine-tuning

epochs = 3  # Adjust as needed

for epoch in range(epochs):
    print(f"\nFine-Tuning Epoch {epoch+1}/{epochs} starting...", flush=True)

    model.train()
    total_loss = 0
    all_labels = []
    all_outputs = []

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=True)

    for batch_idx, batch in enumerate(progress_bar):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Store outputs & labels for metric calculation
        all_labels.append(labels.cpu().detach().numpy())
        all_outputs.append(outputs.cpu().detach().numpy())

        # Update progress bar
        progress_bar.set_postfix({"Loss": loss.item()})

    avg_train_loss = total_loss / len(train_loader)

    # Compute final metrics
    all_labels = np.vstack(all_labels)
    all_outputs = np.vstack(all_outputs)

    auc_score = roc_auc_score(all_labels, all_outputs, average="macro")
    f1 = f1_score(all_labels, all_outputs.round(), average="macro")
    precision = precision_score(all_labels, all_outputs.round(), average="macro", zero_division=0)
    recall = recall_score(all_labels, all_outputs.round(), average="macro", zero_division=0)

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | "
          f"AUC: {auc_score:.4f} | F1-score: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}",
          flush=True)

print("\nFine-Tuning Complete!", flush=True)



Fine-Tuning Epoch 1/3 starting...


Epoch 1/3: 100%|██████████| 1792/1792 [07:59<00:00,  3.73it/s, Loss=0.00734]


Epoch 1/3 | Train Loss: 0.0097 | AUC: 0.9999 | F1-score: 0.9956 | Precision: 0.9951 | Recall: 0.9960

Fine-Tuning Epoch 2/3 starting...


Epoch 2/3: 100%|██████████| 1792/1792 [07:59<00:00,  3.74it/s, Loss=0.00242]


Epoch 2/3 | Train Loss: 0.0069 | AUC: 0.9999 | F1-score: 0.9967 | Precision: 0.9964 | Recall: 0.9971

Fine-Tuning Epoch 3/3 starting...


Epoch 3/3: 100%|██████████| 1792/1792 [07:59<00:00,  3.74it/s, Loss=0.00139]


Epoch 3/3 | Train Loss: 0.0053 | AUC: 1.0000 | F1-score: 0.9973 | Precision: 0.9971 | Recall: 0.9975

Fine-Tuning Complete!


In [ ]:
import os
model_save_dir = "/content/drive/MyDrive/LINKUP_modelDIS_finetuned"
os.makedirs(model_save_dir, exist_ok=True)

model_save_path = os.path.join(model_save_dir, "LINKUP_modelDIS_finetuned.pth")
torch.save(model, model_save_path)
print(f"Fine-Tuned Model saved at {model_save_path}")


Fine-Tuned Model saved at /content/drive/MyDrive/LINKUP_modelDIS_finetuned/LINKUP_modelDIS_finetuned.pth


In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer

# Define device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Path to saved model
model_save_path = "/content/drive/MyDrive/LINKUP_modelDIS_finetuned/LINKUP_modelDIS_finetuned.pth"

# Load the full model
model = torch.load(model_save_path, map_location=device)
model.to(device)
model.eval()  # Set to evaluation mode
print("Fine-tuned model loaded successfully!")

# Load Tokenizer for DistilBERT
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print("Tokenizer loaded!")

# Define MultiLabelClassifier (Same as Training Script)
import torch.nn as nn
from transformers import DistilBertModel

class MultiLabelClassifier(nn.Module):
    def __init__(self, output_dim):
        super(MultiLabelClassifier, self).__init__()
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.fc = nn.Linear(self.bert.config.hidden_size, output_dim)
        self.sigmoid = nn.Sigmoid()  # Multi-label classification

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token representation
        return self.sigmoid(self.fc(pooled_output))  # Convert logits to probabilities

# Load Model
model_save_path = "/content/drive/MyDrive/LINKUP_modelDIS_finetuned/LINKUP_modelDIS_finetuned.pth"
output_dim = 6  # Ensure this matches your number of labels

model = MultiLabelClassifier(output_dim)
model = torch.load(model_save_path, map_location=device)
model.to(device)
model.eval()

print("Fine-Tuned Model loaded successfully!")

# Define Sample Test Sentences
sample_texts = [
    "You are the worst person ever!",
    "I hope you have an amazing day!",
    "I'll kill u!",
    "This is absolute garbage!",
    "What a wonderful place to visit!",
    "I love you",
    "Hello, my name is Anthony",
        "Go to hell!",
    "I hope your day is amazing!",
    "I'm going to destroy you.",
    "You are such a nice person.",
    "Shut up, you idiot.",
    "I disagree, but I respect your opinion.",
    "I hope you die you stupid ugly bitch",
    "you look fat",
    "what size is your dress? XXXXXL?",
    "you're not very smart are you?",
    "how many dogs have you eaten, asian coworker",
    "are you fucking homosexual",
    "coworker, should I kill myself?",
]

# Function to Tokenize Input Texts
def tokenize_texts(texts, tokenizer, max_length=128):
    encoded = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    return encoded["input_ids"], encoded["attention_mask"]

# Tokenize Sample Inputs
X_sample_tokens, X_sample_masks = tokenize_texts(sample_texts, tokenizer)
X_sample_tokens, X_sample_masks = X_sample_tokens.to(device), X_sample_masks.to(device)
print("Tokenization complete! Shape:", X_sample_tokens.shape)

# Run Model on Sample Texts
with torch.no_grad():
    outputs = model(X_sample_tokens, attention_mask=X_sample_masks)  # Get raw logits
    predictions = outputs.cpu().numpy()  # Apply sigmoid activation

print("Processed Predictions:", predictions)  # Debugging Step

# Adjust Threshold to Reduce False Positives
threshold = 0.5  # You can test different thresholds (0.5, 0.7, etc.)
binary_predictions = (predictions >= threshold).astype(int)

# Define Label Categories
label_columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

# Print Results
for i, text in enumerate(sample_texts):
    predicted_labels = [label_columns[j] for j in range(len(label_columns)) if binary_predictions[i][j] == 1]
    print(f"\nText: {text}")
    print("Predicted Categories:", predicted_labels if predicted_labels else ["None"])


<ipython-input-6-bead620436aa>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_save_path, map_location=device)


Fine-tuned model loaded successfully!
Tokenizer loaded!


<ipython-input-6-bead620436aa>:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_save_path, map_location=device)


Fine-Tuned Model loaded successfully!
Tokenization complete! Shape: torch.Size([20, 128])
Processed Predictions: [[9.99902010e-01 5.83732268e-04 8.25856332e-05 1.58088806e-03
  3.99102122e-01 1.40237054e-04]
 [6.35343094e-05 1.24760976e-04 4.28407620e-05 3.61446546e-05
  4.28267173e-04 8.97139762e-05]
 [9.98806596e-01 8.26396572e-04 5.28100471e-04 9.99918938e-01
  4.33454203e-04 1.40810921e-03]
 [9.99767125e-01 2.38211913e-04 2.97127277e-01 1.40648990e-04
  6.19553612e-05 2.29523343e-04]
 [8.47046249e-06 2.46437878e-04 5.54715734e-05 1.68424725e-04
  3.28943646e-03 2.71633558e-04]
 [3.75841931e-03 3.37589619e-04 2.30292295e-04 2.22650182e-04
  4.17579431e-05 5.68618052e-05]
 [4.06776999e-05 9.96789604e-05 2.05902506e-05 1.90286097e-04
  1.50358945e-04 6.28743874e-05]
 [9.99926567e-01 3.22772656e-04 8.26042354e-01 1.24157144e-04
  9.94333386e-01 8.96525762e-06]
 [1.76365502e-05 1.99300353e-04 4.31297558e-05 7.67349338e-05
  6.36283366e-04 2.22271134e-04]
 [9.99707401e-01 2.08261190e-03 

In [ ]:
import torch

quantized_model = torch.quantization.quantize_dynamic(
    model,  # Model to quantize
    {torch.nn.Linear},  # Layers to apply quantization
    dtype=torch.qint8  # Use 8-bit integer for compression
)

# Save the smaller model
quantized_model_path = "/content/drive/MyDrive/LINKUP_modelDIS_quantized.pth"
torch.save(quantized_model.state_dict(), quantized_model_path)
print("Quantized model saved!")


Quantized model saved!


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, DistilBertModel

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Define the Model Structure (Must Match Original)
class MultiLabelClassifier(nn.Module):
    def __init__(self, output_dim):
        super(MultiLabelClassifier, self).__init__()
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.fc = nn.Linear(self.bert.config.hidden_size, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token representation
        return self.sigmoid(self.fc(pooled_output))

# Initialize the Model
output_dim = 6  # Ensure this matches your number of labels
model = MultiLabelClassifier(output_dim)

# Apply Quantization Again (Dynamic Quantization)
quantized_model = torch.quantization.quantize_dynamic(
    model,  # Model to quantize
    {torch.nn.Linear},  # Layers to apply quantization
    dtype=torch.qint8  # Use 8-bit integer for compression
)

# Load the Quantized State Dict
quantized_model_path = "/content/drive/MyDrive/LINKUP_modelDIS_quantized.pth"
quantized_model.load_state_dict(torch.load(quantized_model_path, map_location=device))

# Move to device and set to eval mode
quantized_model.to(device)
quantized_model.eval()

print("Quantized model loaded successfully!")


<ipython-input-11-533a8ccf9129>:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  quantized_model.load_state_dict(torch.load(quantized_model_path, map_location=device))


Quantized model loaded successfully!


In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer
import torch.nn as nn
from transformers import DistilBertModel

# Force Model and Device to CPU
device = torch.device("cpu")

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print("Tokenizer loaded successfully!")

# Define Model Structure
class MultiLabelClassifier(nn.Module):
    def __init__(self, output_dim):
        super(MultiLabelClassifier, self).__init__()
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.fc = nn.Linear(self.bert.config.hidden_size, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        return self.sigmoid(self.fc(pooled_output))

# Initialize Model Structure
output_dim = 6
model = MultiLabelClassifier(output_dim)

# Apply Dynamic Quantization
quantized_model = torch.quantization.quantize_dynamic(
    model,
    {torch.nn.Linear},
    dtype=torch.qint8
)

# Load Quantized Model Weights
quantized_model_path = "/content/drive/MyDrive/LINKUP_final.pth"
quantized_model.load_state_dict(torch.load(quantized_model_path, map_location=device))

# Ensure Model is on CPU and in Evaluation Mode
quantized_model.to(device)
quantized_model.eval()

print("Quantized model loaded successfully!")

# Sample Test Sentences
sample_texts = [
    "You are the worst person ever!",
    "I hope you have an amazing day!",
    "I'll kill u!",
    "This is absolute garbage!",
    "What a wonderful place to visit!",
    "I love you",
    "Hello, my name is Anthony",
    "Go to hell!",
    "I hope your day is amazing!",
    "I'm going to destroy you.",
    "You are such a nice person.",
    "Shut up, you idiot.",
    "I disagree, but I respect your opinion.",
    "I hope you die you stupid ugly bitch",
    "you look fat",
    "what size is your dress? XXXXXL?",
    "you're not very smart are you?",
    "how many dogs have you eaten, asian coworker",
    "are you fucking homosexual",
    "coworker, should I kill myself?",
]

# Function to Tokenize Input Texts
def tokenize_texts(texts, tokenizer, max_length=128):
    encoded = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    return encoded["input_ids"], encoded["attention_mask"]

# Tokenize and Move Inputs to CPU
X_sample_tokens, X_sample_masks = tokenize_texts(sample_texts, tokenizer)
X_sample_tokens, X_sample_masks = X_sample_tokens.to(device), X_sample_masks.to(device)

# Print Tokenization Shape
print(f"Tokenization complete! Shape: {X_sample_tokens.shape}")

# Run Model on Sample Texts
with torch.no_grad():
    outputs = quantized_model(X_sample_tokens, attention_mask=X_sample_masks)
    predictions = outputs.cpu().numpy()

# Print Processed Predictions
print("Processed Predictions:", predictions)

# Apply Threshold for Classification
threshold = 0.5
binary_predictions = (predictions >= threshold).astype(int)

# Define Label Categories
label_columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

# Print Results
for i, text in enumerate(sample_texts):
    predicted_labels = [label_columns[j] for j in range(len(label_columns)) if binary_predictions[i][j] == 1]
    print(f"\nText: {text}")
    print("Predicted Categories:", predicted_labels if predicted_labels else ["None"])


Tokenizer loaded successfully!


/usr/local/lib/python3.11/dist-packages/torch/_utils.py:410: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


Quantized model loaded successfully!
Tokenization complete! Shape: torch.Size([22, 128])
Processed Predictions: [[9.9923337e-01 1.2398855e-04 1.6992586e-04 7.3362282e-04 8.7648499e-01
  3.2024658e-05]
 [3.0926251e-04 1.6203623e-04 1.2005849e-04 3.3619934e-05 5.4767943e-04
  9.7570613e-05]
 [9.9875534e-01 3.5899217e-04 4.2883487e-04 9.9368852e-01 6.9979258e-05
  7.7432889e-04]
 [9.9904591e-01 3.0691741e-04 7.6916441e-02 9.6278432e-05 5.7004978e-05
  1.4748689e-04]
 [3.8707658e-06 3.6496215e-04 1.2562229e-04 2.5409050e-04 8.6261109e-03
  6.5809663e-04]
 [3.7465950e-03 2.9112294e-04 6.0185831e-04 1.1194328e-04 1.0241200e-04
  1.2025934e-04]
 [6.8944551e-05 8.8962253e-05 2.5293037e-05 1.4394987e-04 2.7484904e-04
  6.5459331e-05]
 [9.9825901e-01 1.4333529e-04 4.5933932e-01 9.9410427e-05 9.6026474e-01
  2.1643129e-05]
 [9.9180434e-05 1.2967225e-04 3.6977217e-05 4.8254908e-05 2.4858158e-04
  1.7467287e-04]
 [9.9967432e-01 7.9729309e-04 2.6794072e-04 9.6024060e-01 1.9639394e-04
  5.6550687e-04